# Import libraries

In [1]:
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import trange
from torch.autograd import Variable
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
import gensim.downloader

In [2]:
word2vec_goog1e_news: gensim.models.keyedvectors.KeyedVectors = gensim.downloader.load('word2vec-google-news-300')
word2vec_goog1e_news.add_vector("<pad>", np.zeros(300))
pad_index = word2vec_goog1e_news.key_to_index["<pad>"]
embedding_weights = torch.FloatTensor(word2vec_goog1e_news.vectors)
vocab = word2vec_goog1e_news.key_to_index

KeyboardInterrupt: 

# Import Dataset

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
def tokenize_pd_series_to_lsit(list_of_text):
    tokenized = []
    for sentence in list_of_text:
        tokenized.append(word_tokenize(sentence.lower()))
    return tokenized

def format_label(label):
    return torch.unsqueeze(torch.tensor(label.to_list()), axis=1).tolist()

def indexify(data):
    setences = []
    for sentence in data:
        s = [vocab[token] if token in vocab
            else vocab['UNK']
            for token in sentence]
        setences.append(s)
    return setences

In [ ]:
# For debugging
training_data = pd.read_csv(filepath_or_buffer="TREC_dataset/modified_training_data.csv", sep=",") 
test_data = pd.read_csv(filepath_or_buffer="TREC_dataset/modified_test_data.csv", sep=",")
# Actual data
# training_data = pd.read_csv(filepath_or_buffer="TREC_dataset/eugene_train.csv", sep=",") 
# test_data = pd.read_csv(filepath_or_buffer="TREC_dataset/eugene_test.csv", sep=",")

X = training_data["text"]
y = training_data["label-coarse"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=500)

X_test = test_data["text"]
y_test = test_data["label-coarse"]

X_train_lst = X_train.to_list()
X_val_lst = X_val.to_list()
X_test_lst = X_test.to_list()

X_train_tokenized = tokenize_pd_series_to_lsit(X_train_lst)
X_val_tokenized = tokenize_pd_series_to_lsit(X_val_lst)
X_test_tokenized = tokenize_pd_series_to_lsit(X_test_lst)

no_of_labels = max(y_train.to_list()) + 1

In [ ]:
X_train_tokenized_indexified = indexify(X_train_tokenized)
X_val_tokenized_indexified = indexify(X_val_tokenized)
X_test_tokenized_indexified = indexify(X_test_tokenized)

y_train_formatted = format_label(y_train)
y_val_formatted = format_label(y_val)
y_test_formatted = format_label(y_test)

In [ ]:
def data_iterator(sentences, labels, total_size: int, batch_size: int, shuffle: bool=False):
    # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
    order = list(range(total_size))
    if shuffle:
        random.seed(230)
        random.shuffle(order)

    # one pass over data
    for i in range((total_size+1)//batch_size):
        # fetch sentences and tags
        batch_sentences = [sentences[idx] for idx in order[i*batch_size:(i+1)*batch_size]]
        batch_tags = [labels[idx] for idx in order[i*batch_size:(i+1)*batch_size]]

        # compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in batch_sentences])

        # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
        # initialising labels to -1 differentiates tokens with tags from PADding tokens
        batch_data = vocab['<pad>']*np.ones((len(batch_sentences), batch_max_len))
        batch_labels = -1*np.ones(len(batch_sentences))
        batch_labels = batch_labels.reshape(-1, 1)
        # For debugging
        # print(f"batch_data.shape = {batch_data.shape}")
        # print(f"batch_labels.shape = {batch_labels.shape}")

        # copy the data to the numpy array
        for j in range(len(batch_sentences)):
            cur_len = len(batch_sentences[j])
            batch_data[j][:cur_len] = batch_sentences[j]
            batch_labels[j][:cur_len] = batch_tags[j]

        # since all data are indices, we convert them to torch LongTensors
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)
        
        # convert them to Variables to record operations in the computational graph
        batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)

        yield batch_data, batch_labels, batch_sentences

In [ ]:
class Net(nn.Module):
    """
    This is the standard way to define your own network in PyTorch. You typically choose the components
    (e.g. LSTMs, linear layers etc.) of your network in the __init__ function. You then apply these layers
    on the input step-by-step in the forward function. You can use torch.nn.functional to apply functions
    such as F.relu, F.sigmoid, F.softmax. Be careful to ensure your dimensions are correct after each step.

    You are encouraged to have a look at the network in pytorch/vision/model/net.py to get a better sense of how
    you can go about defining your own network.

    The documentation for all the various components available to you is here: http://pytorch.org/docs/master/nn.html
    """

    def __init__(self, embedding_weights, embedding_dim, lstm_hidden_dim, number_of_tags):
        """
        We define an recurrent network that predicts the NER tags for each token in the sentence. The components
        required are:

        - an embedding layer: this layer maps each index in range(params.vocab_size) to a params.embedding_dim vector
        - lstm: applying the LSTM on the sequential input returns an output for each token in the sentence
        - fc: a fully connected layer that converts the LSTM output for each token to a distribution over NER tags

        Args:
            params: (Params) contains vocab_size, embedding_dim, lstm_hidden_dim
        """
        super(Net, self).__init__()

        # the embedding takes as input the vocab_size and the embedding_dim
        # self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, padding_idx=pad_index)

        # the LSTM takes as input the size of its input (embedding_dim), its hidden size
        # for more details on how to use it, check out the documentation
        self.lstm = nn.LSTM(embedding_dim,
                            lstm_hidden_dim, batch_first=True)

        # the fully connected layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, number_of_tags)

    def forward(self, s):
        """
        This function defines how we use the components of our network to operate on an input batch.

        Args:
            s: (Variable) contains a batch of sentences, of dimension batch_size x seq_len, where seq_len is
               the length of the longest sentence in the batch. For sentences shorter than seq_len, the remaining
               tokens are PADding tokens. Each row is a sentence with each element corresponding to the index of
               the token in the vocab.

        Returns:
            out: (Variable) dimension batch_size*seq_len x num_tags with the log probabilities of tokens for each token
                 of each sentence.

        Note: the dimensions after each step are provided
        """
        #                                -> batch_size x seq_len
        # apply the embedding layer that maps each token to its embedding
        # dim: batch_size x seq_len x embedding_dim
        s = self.embedding(s)

        # run the LSTM along the sentences of length seq_len
        # dim: batch_size x seq_len x lstm_hidden_dim
        s, _ = self.lstm(s)

        # Changed
        s = torch.mean(s, dim=1)  # mean pooling

        # apply the fully connected layer and obtain the output (before softmax) for each token
        s = self.fc(s)                   # dim: batch_size*seq_len x num_tags

        # apply log softmax on each token's output (this is recommended over applying softmax
        # since it is numerically more stable)
        return F.log_softmax(s, dim=1)   # dim: batch_size*seq_len x num_tags

In [ ]:
def accuracy(outputs, labels):
    outputs = np.argmax(outputs.detach().numpy(), axis=1)
    # compare outputs with labels
    labels = labels.squeeze()
    # For debugging
    # print(f"{outputs=}")
    # print(f"{labels=}")
    return np.sum([1 if first == second else 0 for first, second in zip(labels, outputs)]) / float(len(labels))

def loss_fn(outputs, labels):
    """
    Compute the cross entropy loss given outputs from the model and labels for all tokens. Exclude loss terms
    for PADding tokens.

    Args:
        outputs: (Variable) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (Variable) dimension batch_size x seq_len where each element is either a label in [0, 1, ... num_tag-1],
                or -1 in case it is a PADding token.

    Returns:
        loss: (Variable) cross entropy loss for all tokens in the batch

    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """

    loss = F.cross_entropy(outputs, labels.squeeze())
    return loss

In [ ]:
class RunningAverage:
    """A simple class that maintains the running average of a quantity

    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)

In [ ]:
def train(model, optimizer, loss_fn, data_iterator, metrics, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch, _ = next(data_iterator)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % 10 == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

        # update the average loss
        loss_avg.update(loss.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

In [ ]:
def evaluate(model, loss_fn, data_iterator, metrics, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch, _ = next(data_iterator)

        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)
        accurary1 = accuracy(output_batch, labels_batch)
        print(f"{accurary1=}")

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

In [ ]:
def train_and_evaluate(
        model,
        train_sentences,
        train_labels,
        val_sentences,
        val_labels,
        num_epochs: int,
        batch_size: int,
        optimizer,
        loss_fn,
        metrics
):
    for epoch in range(num_epochs):
        # Run one epoch
        print("Epoch {}/{}".format(epoch + 1, num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (len(train_sentences) + 1) // batch_size
        train_data_iterator = data_iterator(
            train_sentences, train_labels, len(train_sentences), batch_size, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator,
              metrics, num_steps)

        # Evaluate for one epoch on validation set
        num_steps = (len(val_sentences) + 1) // batch_size
        val_data_iterator = data_iterator(
            val_sentences, val_labels, len(val_sentences), batch_size, shuffle=True)
        val_metrics = evaluate(
            model, loss_fn, val_data_iterator, metrics, num_steps)
   

In [ ]:
inv_vocab = {v: k for k, v in vocab.items()}

def id_to_words(sentence):
    new_sentence = [inv_vocab[i] for i in sentence]
    return new_sentence

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# manually change vocab size (unique no. of words) and change label size (unique no. of labels) for now
model = Net(embedding_weights, 300, 300, no_of_labels)
optimizer = optim.Adam(model.parameters(), lr=0.01)
train_and_evaluate(model, X_train_tokenized_indexified , y_train_formatted , X_val_tokenized_indexified  , y_val_formatted , 30, 64, optimizer, loss_fn, metrics)

Epoch 1/30


100%|██████████| 77/77 [00:03<00:00, 21.81it/s, loss=1.112]


outputs=array([0, 0, 0, 4, 1, 4, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0,
       0, 4, 0, 0, 1, 4, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 0, 4, 4, 0, 3, 3,
       4, 4, 4, 1, 3, 3, 0, 1, 0, 3, 0, 3, 4, 0, 0, 0, 4, 3, 0, 3],
      dtype=int64)
labels=tensor([0, 0, 0, 4, 1, 3, 3, 3, 1, 1, 1, 1, 1, 2, 0, 4, 3, 1, 0, 3, 4, 1, 0, 4,
        0, 1, 1, 1, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 1, 4, 0, 0, 3, 3, 1, 4, 4, 1,
        3, 3, 0, 2, 0, 3, 1, 3, 4, 0, 4, 0, 4, 3, 0, 3])
accurary1=0.703125
outputs=array([0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 4, 4, 4, 0, 4, 0, 1, 0, 0, 3, 4, 0,
       1, 3, 0, 0, 3, 4, 0, 0, 4, 4, 4, 0, 4, 4, 0, 4, 3, 4, 0, 3, 0, 0,
       0, 0, 1, 1, 0, 0, 4, 0, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 3],
      dtype=int64)
labels=tensor([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 1, 1, 0, 3, 4, 1, 1, 3,
        3, 1, 3, 0, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0, 0, 3, 1, 1,
        1, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1])
accurary1=0.734375
outputs=array([4, 3, 4, 0, 4, 4,

100%|██████████| 77/77 [00:03<00:00, 22.44it/s, loss=0.617]


outputs=array([0, 0, 0, 4, 1, 3, 3, 3, 0, 1, 0, 1, 0, 1, 0, 0, 3, 0, 0, 3, 4, 0,
       0, 4, 0, 0, 0, 1, 4, 0, 1, 0, 4, 4, 4, 1, 4, 1, 0, 4, 4, 0, 3, 3,
       4, 4, 4, 1, 3, 3, 0, 0, 0, 3, 0, 3, 4, 0, 4, 0, 4, 3, 0, 3],
      dtype=int64)
labels=tensor([0, 0, 0, 4, 1, 3, 3, 3, 1, 1, 1, 1, 1, 2, 0, 4, 3, 1, 0, 3, 4, 1, 0, 4,
        0, 1, 1, 1, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 1, 4, 0, 0, 3, 3, 1, 4, 4, 1,
        3, 3, 0, 2, 0, 3, 1, 3, 4, 0, 4, 0, 4, 3, 0, 3])
accurary1=0.765625
outputs=array([0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 4, 4, 4, 0, 1, 0, 1, 0, 0, 3, 4, 1,
       1, 3, 0, 1, 3, 4, 0, 0, 4, 1, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 0, 0,
       0, 3, 1, 1, 0, 0, 4, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 3],
      dtype=int64)
labels=tensor([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 1, 1, 0, 3, 4, 1, 1, 3,
        3, 1, 3, 0, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0, 0, 3, 1, 1,
        1, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1])
accurary1=0.8125
outputs=array([4, 3, 4, 0, 4, 4, 0

...

outputs=array([0, 0, 0, 4, 1, 3, 3, 3, 0, 1, 1, 1, 1, 0, 0, 4, 1, 1, 0, 3, 4, 4,
       0, 4, 1, 0, 1, 1, 4, 0, 1, 0, 4, 4, 4, 1, 4, 1, 1, 4, 4, 0, 3, 3,
       4, 4, 4, 1, 3, 3, 0, 2, 0, 3, 0, 3, 4, 0, 4, 2, 4, 3, 0, 3],
      dtype=int64)
labels=tensor([0, 0, 0, 4, 1, 3, 3, 3, 1, 1, 1, 1, 1, 2, 0, 4, 3, 1, 0, 3, 4, 1, 0, 4,
        0, 1, 1, 1, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 1, 4, 0, 0, 3, 3, 1, 4, 4, 1,
        3, 3, 0, 2, 0, 3, 1, 3, 4, 0, 4, 0, 4, 3, 0, 3])
accurary1=0.828125
outputs=array([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 1, 1, 1, 0, 3, 4, 1,
       1, 3, 3, 1, 3, 4, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0,
       0, 3, 0, 1, 0, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1],
      dtype=int64)
labels=tensor([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 1, 1, 0, 3, 4, 1, 1, 3,
        3, 1, 3, 0, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0, 0, 3, 1, 1,
        1, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1])
accurary1=0.9375
outputs=array([4, 3, 4, 0, 4, 4, 1

100%|██████████| 77/77 [00:04<00:00, 18.52it/s, loss=0.004]


outputs=array([0, 0, 0, 4, 1, 3, 3, 3, 0, 1, 1, 1, 1, 0, 0, 4, 1, 1, 0, 3, 4, 4,
       0, 4, 1, 0, 1, 1, 4, 0, 1, 0, 4, 4, 4, 1, 4, 1, 1, 4, 4, 0, 3, 3,
       4, 4, 4, 1, 3, 3, 0, 2, 0, 3, 0, 3, 4, 0, 4, 2, 4, 3, 0, 3],
      dtype=int64)
labels=tensor([0, 0, 0, 4, 1, 3, 3, 3, 1, 1, 1, 1, 1, 2, 0, 4, 3, 1, 0, 3, 4, 1, 0, 4,
        0, 1, 1, 1, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 1, 4, 0, 0, 3, 3, 1, 4, 4, 1,
        3, 3, 0, 2, 0, 3, 1, 3, 4, 0, 4, 0, 4, 3, 0, 3])
accurary1=0.828125
outputs=array([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 1, 1, 1, 0, 3, 4, 1,
       1, 3, 3, 1, 3, 4, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0,
       0, 3, 0, 1, 0, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1],
      dtype=int64)
labels=tensor([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 1, 1, 0, 3, 4, 1, 1, 3,
        3, 1, 3, 0, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0, 0, 3, 1, 1,
        1, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1])
accurary1=0.9375
outputs=array([4, 3, 4, 0, 4, 4, 1

100%|██████████| 77/77 [00:04<00:00, 17.43it/s, loss=0.003]


outputs=array([0, 0, 0, 4, 1, 3, 3, 3, 0, 1, 1, 1, 1, 0, 0, 4, 1, 1, 0, 3, 4, 4,
       0, 4, 1, 0, 1, 1, 4, 0, 1, 0, 4, 4, 4, 1, 4, 1, 1, 4, 4, 0, 3, 3,
       4, 4, 4, 1, 3, 3, 0, 2, 0, 3, 0, 3, 4, 0, 4, 2, 4, 3, 0, 3],
      dtype=int64)
labels=tensor([0, 0, 0, 4, 1, 3, 3, 3, 1, 1, 1, 1, 1, 2, 0, 4, 3, 1, 0, 3, 4, 1, 0, 4,
        0, 1, 1, 1, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 1, 4, 0, 0, 3, 3, 1, 4, 4, 1,
        3, 3, 0, 2, 0, 3, 1, 3, 4, 0, 4, 0, 4, 3, 0, 3])
accurary1=0.828125
outputs=array([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 1, 1, 1, 0, 3, 4, 1,
       1, 3, 3, 1, 3, 4, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0,
       0, 3, 0, 1, 0, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1],
      dtype=int64)
labels=tensor([0, 4, 2, 0, 4, 1, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 1, 1, 0, 3, 4, 1, 1, 3,
        3, 1, 3, 0, 0, 0, 4, 3, 1, 0, 1, 4, 0, 4, 3, 4, 0, 3, 1, 0, 0, 3, 1, 1,
        1, 0, 1, 3, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4, 1])
accurary1=0.9375
outputs=array([4, 3, 4, 0, 4, 4, 1